In [20]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.master('local').getOrCreate()
#sc = SparkContext().getOrCreate()
from pyspark.sql import SQLContext

In [21]:
import pandas as pd
import pickle
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col

In [22]:
tweets = pd.read_pickle('/Users/matthewcoleman/Desktop/hate_speech_classifier/data/labeled_data.p')

In [23]:
tweets_full = spark.createDataFrame(tweets)

In [24]:
tweets_full.show(5)

+-----+-----------+------------------+-------+-----+--------------------+
|count|hate_speech|offensive_language|neither|class|               tweet|
+-----+-----------+------------------+-------+-----+--------------------+
|    3|          0|                 0|      3|    2|!!! RT @mayasolov...|
|    3|          0|                 3|      0|    1|!!!!! RT @mleew17...|
|    3|          0|                 3|      0|    1|!!!!!!! RT @UrKin...|
|    3|          0|                 2|      1|    1|!!!!!!!!! RT @C_G...|
|    6|          0|                 6|      0|    1|!!!!!!!!!!!!! RT ...|
+-----+-----------+------------------+-------+-----+--------------------+
only showing top 5 rows



Class labels are labeled as follows:
* 0 - hate speech 
* 1 - offensive language 
* 2 - neither

In [25]:
tweets = tweets_full.select('tweet', col('class').alias('label'))
tweets.show(5)

+--------------------+-----+
|               tweet|label|
+--------------------+-----+
|!!! RT @mayasolov...|    2|
|!!!!! RT @mleew17...|    1|
|!!!!!!! RT @UrKin...|    1|
|!!!!!!!!! RT @C_G...|    1|
|!!!!!!!!!!!!! RT ...|    1|
+--------------------+-----+
only showing top 5 rows



In [26]:
#Configuring ML pipeline
englishStopWords = StopWordsRemover.loadDefaultStopWords('english').append('"')

#Transformers
tokenizer = Tokenizer(inputCol = 'tweet',
                     outputCol = 'words')
stops = StopWordsRemover(inputCol = 'words', outputCol = 'words_clean',stopWords = englishStopWords)
hashingTF = HashingTF(inputCol = stops.getOutputCol(),
                     outputCol = 'features')



#Estimator
lr = LogisticRegression(maxIter = 10, regParam = .1)

#Stages
stages = [tokenizer, stops, hashingTF, lr]

#Pipeline 

pl = Pipeline(stages = stages)

In [27]:
#Train Test Split the data 
train, test = tweets.randomSplit([.75,.25], seed = 4292020)

In [28]:
# training the pipeline
model = pl.fit(train)

Py4JJavaError: An error occurred while calling o247.transform.
: java.lang.NullPointerException
	at scala.collection.mutable.ArrayOps$ofRef$.length$extension(ArrayOps.scala:192)
	at scala.collection.mutable.ArrayOps$ofRef.length(ArrayOps.scala:192)
	at scala.collection.SeqLike$class.size(SeqLike.scala:106)
	at scala.collection.mutable.ArrayOps$ofRef.size(ArrayOps.scala:186)
	at scala.collection.mutable.Builder$class.sizeHint(Builder.scala:69)
	at scala.collection.mutable.ArrayBuilder.sizeHint(ArrayBuilder.scala:22)
	at scala.collection.TraversableLike$class.builder$1(TraversableLike.scala:230)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:233)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
	at org.apache.spark.ml.feature.StopWordsRemover.transform(StopWordsRemover.scala:122)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.Delegati

In [ ]:
model_train = model.transform(train)
#model_train.show()
model_train.stat.crosstab('label','prediction').show()

In [ ]:
model_train.select('words').show()

In [ ]:
pd_model = model_train.toPandas()

In [29]:
pd_model[(pd_model['label'] == 0) & (pd_model['prediction']==1)] #.loc[2229]['words']

NameError: name 'pd_model' is not defined

In [ ]:
model_test = model.transform(test)
model_test.stat.crosstab('label','prediction').show()

In [72]:
spark.stop()
#sc.stop() 